In [65]:
import os
import pandas as pd 
from datetime import date, timedelta, datetime
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

os.getcwd()


'/Users/marekerben/Desktop/Prakticka'

In [66]:
def date_range_list(start_date, end_date):
    list_of_dates = []
    # Return generator for a list datetime.date objects (inclusive) between start_date and end_date (inclusive).
    curr_date = start_date
    while curr_date <= end_date:
        list_of_dates.append(curr_date.strftime('%Y-%m-%d'))
        curr_date += timedelta(days=1)
    return list_of_dates

def list_of_halfhour_timestamps():
    times = []
    for hour in range(24):
        for minute in ['00', '30']:
            time_str = f"{hour:02d}:{minute}"
            times.append(time_str)
    return times


def timestamp_to_UTC(series):
    timestamp = series / 1000000
    utc_timestamp = []
    for i in range(len(timestamp)):
        utc_datetime = datetime.utcfromtimestamp(timestamp[i])
        utc_timestamp.append(utc_datetime)
    return utc_timestamp

def calculate_delta_mid_price_and_ofi(datum):
    quotes_df = pd.read_csv(f'/Users/marekerben/Desktop/Prakticka/binance-futures/BTCUSDT/quotes/{datum}.csv.gz')
    quotes_df['UTC'] = timestamp_to_UTC(quotes_df['timestamp'])
    
    tick_size = 1
    
    quotes_df['mid_price'] = (quotes_df['ask_price'] + quotes_df['bid_price']) / (2 * tick_size)
    
    nrows = quotes_df.shape[0]
    e = [None]

    for n in range(1, nrows):
        e.append(quotes_df['bid_amount'][n] * (quotes_df['bid_price'][n] >= quotes_df['bid_price'][n-1]) 
                 - quotes_df['bid_amount'][n-1] * (quotes_df['bid_price'][n] <= quotes_df['bid_price'][n-1])
                 - quotes_df['ask_amount'][n] * (quotes_df['ask_price'][n] <= quotes_df['ask_price'][n-1])
                 + quotes_df['ask_amount'][n-1] * (quotes_df['ask_price'][n] >= quotes_df['ask_price'][n-1])
                 )

    quotes_df['e_n'] = e

    quotes_df.set_index('UTC', inplace = True)

    return pd.DataFrame({'delta_midprice': quotes_df.mid_price.resample('10S').apply(lambda x: x[x.last_valid_index()] - x[x.first_valid_index()]),
                               'OFI':quotes_df.e_n.resample('10S').sum()})


In [67]:
#Generate list of dates that I will be looping through
sdate = date(year = 2020, month = 11, day = 15)
edate = date(year = 2022, month = 2, day = 15)

date_list = date_range_list(sdate, edate)
print(len(date_list))

#Generating list of collumn names that I will be using in the output file
hour_list = list_of_halfhour_timestamps()
hour_list.insert(0, "date")

458


In [72]:
#Computing data frames for each day and storing them in dictionary
delta_midprice_and_ofi_dfs = {}
for i in ['2020-11-15', '2020-11-16', '2020-11-17']: #date_list
    delta_midprice_and_ofi_dfs[i] = calculate_delta_mid_price_and_ofi(i)

In [32]:
#Uložení delta_midprice a OFI, abych to nemusel počítat znova
'''
delta_midprice_and_ofi_dfs['2020-11-15'].to_csv(f'{list(seznam.keys())[0]}.csv')
tohle ještě nebudu ukládat, protože nevím, jestli to je úplně správně. 
'''

"\nseznam['2020-11-15'].to_csv(f'{list(seznam.keys())[0]}.csv')\ntohle ještě nebudu ukládat, protože nevím, jestli to je úplně správně. \n"

In [74]:
#Writing a output file where on each line are 48 values of beta1 for each day 
with open('Output_file.csv', 'a') as f:
    f.write(",".join(hour_list) + "\n")
    for i in delta_midprice_and_ofi_dfs.keys():
        delta_midprice_and_ofi_dfs[i]['Interval'] = delta_midprice_and_ofi_dfs[i].index.floor('30T')
        regressions = list(delta_midprice_and_ofi_dfs[i].groupby('Interval').apply(lambda x: sm.formula.ols('delta_midprice ~ OFI', data = x).fit())) 
        results = [i,]
        for j in regressions:
            results.append(str(j.params['OFI']))
        f.write(",".join(results) + "\n")

In [75]:
df = pd.read_csv('Output_file.csv')
df


,date,00:00,00:30,01:00,01:30,02:00,02:30,03:00,03:30,04:00,...,19:00,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30
0,2020-11-15,0.085113,0.035403,0.048626,0.045937,0.043114,0.043025,0.042343,0.034960,0.034240,...,0.029585,0.033527,0.032171,0.031951,0.025852,0.026852,0.036588,0.029561,0.050323,0.048392
1,2020-11-16,0.039032,0.036972,0.044404,0.028601,0.037174,0.029038,0.034886,0.038775,0.033145,...,0.067445,0.061006,0.057551,0.049085,0.045944,0.083527,0.069411,0.058078,0.049700,0.061703
2,2020-11-17,0.038024,0.047933,0.041687,0.042335,0.038188,0.035148,0.042169,0.039229,0.037001,...,0.111687,0.082722,0.092457,0.069692,0.054524,0.075495,0.057990,0.049046,0.055654,0.062044


In [76]:
#zatím mi ani jedna beta nevyšla záporně
sum(df.describe().min() < 0)

for i in regressions:
    print(i.summary())

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         delta_midprice   R-squared:                       0.516
Model:                            OLS   Adj. R-squared:                  0.514
Method:                 Least Squares   F-statistic:                     190.0
Date:                Mon, 03 Apr 2023   Prob (F-statistic):           7.00e-30
Time:                        18:06:04   Log-Likelihood:                -476.37
No. Observations:                 180   AIC:                             956.7
Df Residuals:                     178   BIC:                             963.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3088      0.259     -1.193      0.235      -0.820       0.202
OFI            0.0380      0.003     13.784      0.000       0.033       0.043
==============================================================================
Omnibus:                       28.796   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               99.720
Skew:                           0.539   Prob(JB):                     2.22e-22
Kurtosis:                       6.483   Cond. No.                         95.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
#zatím nevyužité z předchozího notebooku
'''
#controlling for NAs
print(quotes_df.isna().sum())

delta_midprice_and_ofi_dfs[<datum>].resample('30T').OFI.agg([np.mean, np.std])

deltaPrice_OFI.resample('30T').OFI.agg([np.mean, np.std]).plot()
plt.show()
plt.savefig('15.11.2020_OFI_mean_std.png')

URL = 'https://sobesice-capital-tardis.s3.eu-central-1.amazonaws.com/{exchange}/{symbol}/{dataType}/{date}.csv.gz'
URL.format(exchange = 'binance-futures',
           symbol = 'BTCUSDT',
           dataType = 'quotes',
           date = '2020-11-15')
'''